In [ ]:
pip install geopandas networkx torch torch-geometric tqdm


In [ ]:
import geopandas as gpd
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from shapely.geometry import Point, LineString
import numpy as np
from tqdm import tqdm

# Step 1: Load Shapefile & Build Graph
def shapefile_to_pyg(shapefile_path):
    gdf = gpd.read_file(shapefile_path).to_crs(epsg=4326)
    G = nx.Graph()
    for idx, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
        geom = row.geometry
        if isinstance(geom, LineString):
            coords = list(geom.coords)
            for i in range(len(coords) - 1):
                u, v = coords[i], coords[i+1]
                for node in [u, v]:
                    if node not in G:
                        G.add_node(node, x=node[0], y=node[1])
                dist = Point(u).distance(Point(v))
                G.add_edge(u, v, distance=dist)
    node_list = list(G.nodes)
    node_indices = {node: i for i, node in enumerate(node_list)}
    x = torch.tensor([[G.nodes[node]['x'], G.nodes[node]['y']] for node in node_list], dtype=torch.float)
    edge_index = []
    edge_attr = []
    for u, v, data in G.edges(data=True):
        edge_index.append([node_indices[u], node_indices[v]])
        edge_index.append([node_indices[v], node_indices[u]])
        edge_attr.append([data['distance']])
        edge_attr.append([data['distance']])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    y = torch.tensor([G.degree[node] for node in node_list], dtype=torch.float).unsqueeze(1)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

# Step 2: Define GCN Model
class GCNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Step 3: Training Loop
def train(model, data, epochs=100, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.mse_loss(out, data.y)
        loss.backward()
        optimizer.step()
        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")

# Step 4: Putting It All Together
if __name__ == "__main__":
    shapefile_path = 'your_shapefile.shp'
    data = shapefile_to_pyg(shapefile_path)
    model = GCNModel(in_channels=2, hidden_channels=16, out_channels=1)
    train(model, data, epochs=50, lr=0.01)
    # Inference
    model.eval()
    predictions = model(data.x, data.edge_index).detach()
    print("\nSample Predictions (first 5 nodes):")
    print(predictions[:5])
